In [2]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import Layout, widgets
from IPython.display import display, clear_output
import scipy
from scipy import signal
import requests
import warnings
warnings.filterwarnings('ignore')

# Replace the URL with the URL of the raw content
url = "https://raw.githubusercontent.com/ntua-el17840/Interactive-Digital-Communications/main/test-book/_static/sima.txt"
response = requests.get(url)

# Split the response text by new lines and convert each line to a float
s = np.array([float(line) for line in response.text.splitlines()])

# Define the sampling frequency
Fs = 8192

# Calculate the power spectral density using Welch's method
f, Pxx = scipy.signal.welch(s, Fs)

# Define filter length and frequency response
N = Fs 
H = np.concatenate((np.ones(N//8), np.zeros(N//4), np.ones(N//8)))
h = np.fft.ifft(H, n=N).real
h = np.fft.fftshift(h)

# Extract filter taps for different window sizes
h32 = h[N//2-16:N//2+17]
h64 = h[N//2-32:N//2+33]
h128 = h[N//2-64:N//2+65]

# Assuming H is defined and calculated as before
H = np.hstack((np.ones(int(Fs/8)), np.zeros(int(Fs-Fs/4)), np.ones(int(Fs/8))))
h = np.real(np.fft.ifft(H))
middle = int(len(h)/2)
h = np.hstack((h[middle:], h[:middle]))

# Extract filter taps for different window sizes
h32 = h[middle-16:middle+16]
h64 = h[middle-32:middle+32]
h128 = h[middle-64:middle+64]
h140 = h[middle-70:middle+70]
h256 = h[middle-128:middle+128]

# Compute frequency responses for the filters
freq32, resp32 = signal.freqz(h32)
freq64, resp64 = signal.freqz(h64)
freq128, resp128 = signal.freqz(h128)
freq140, resp160 = signal.freqz(h140)
freq256, resp256 = signal.freqz(h256)

# Function to apply window and compute frequency responses
def compute_filtered_freq_responses(window_type='Rectangular'):
    freqs, resps = {}, {}
    for filt_size in [32, 64, 128, 140, 256]:
        filt_name = f'h{filt_size}'
        filt = eval(filt_name)
        
        # Apply the selected window type
        if window_type == 'Hamming':
            filt = filt * np.hamming(filt_size)
        elif window_type == 'Kaiser':
            beta = 14  # Example beta value for Kaiser window, adjust as needed
            filt = filt * np.kaiser(filt_size, beta)
            
        freqs[filt_name], resps[filt_name] = signal.freqz(filt)
    return freqs, resps

# Initial computation with Rectangular (no window)
freqs, resps = compute_filtered_freq_responses()

# Output widget for displaying plots
output = widgets.Output()
output.layout = Layout(width='auto', margin='0 auto')

# Function to update the plot based on filter selection and window type
def update_plot(change=None):
    window_type = window_type_dropdown.value
    freqs, resps = compute_filtered_freq_responses(window_type)
    
    # Clear the current output and update the plot
    with output:
        clear_output(wait=True)
        plt.figure(figsize=(10, 5))
        for filt in ['h32', 'h64', 'h128', 'h140', 'h256']:
            if checkboxes[filt].value:
                w, h = freqs[filt], resps[filt]
                plt.plot(0.5 * Fs * w / np.pi, 20 * np.log10(np.abs(h)), label=filt)
        plt.title(f'Frequency Response with {window_type} Window')
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('Magnitude (dB)')
        plt.xscale('linear')
        plt.grid(True)
        plt.legend()

        plt.show()

# Create checkboxes for each filter
checkboxes = {f'h{size}': widgets.Checkbox(value=True, description=f'h{size}') for size in [32, 64, 128, 140, 256]}
for cb in checkboxes.values():
    cb.observe(update_plot, names='value')

# Dropdown for selecting the window type
window_type_dropdown = widgets.Dropdown(
    options=['Rectangular', 'Hamming'],
    value='Rectangular',
    description='Window Type:',
    style={'description_width': 'initial'}
)
window_type_dropdown.observe(update_plot, names='value')

# VBox layout to align items
vbox_layout = Layout(align_items='center', justify_content='center')
vbox_checkboxes = widgets.VBox(list(checkboxes.values()) + [window_type_dropdown], layout=vbox_layout)

# Combine checkboxes and dropdown into a final layout
vbox_loader = widgets.HBox([vbox_checkboxes])

# Combine everything into a final VBox with the layout
vbox_final = widgets.VBox([vbox_loader, output], layout=vbox_layout)

# Display the final layout
display(vbox_final)

# Initial plot update to show default visualization
update_plot()
